# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 郭台銘欣賞學生AI節能創意！霸氣喊「三創空調包給你」　頒發「鴻海獎學鯨」3年逾億元

▲郭台銘出席第三屆鴻海獎學鯨頒獎典禮。（圖／記者林敬旻攝）
記者姚惠茹／台北報導
鴻海（2317）創辦人郭台銘今（2）日親自出席「第三屆鴻海獎學鯨」頒獎典禮，今年首度新增碩博土生名額，共計頒發330位得獎者，每位10萬元獎學金，總計3年累積1,449名學生，總獎學金逾億元，其中有清貧學子用AI打造節省能源動態決策系統，郭台銘更是霸氣喊出「三創空調包給你」。
郭台銘表示，施比受更有福，看到學生們不受艱困環境影響來奮鬥的過程，很為學生感到驕傲，希望「鴻海獎學鯨」不只是給予資源上的幫助，還能讓學生利用這些資源，擁有自信與機會來發展天賦、追求夢想與實現自我，並強調「每個人都可以超越郭台銘」。
其中目前就讀於台北科技大學的廖健亦，因為父親受傷失業、母親罹患癌症，所以都是透過他積極打工和申請獎學金來改善家中困境，但仍努力考取專業級國際認證「冷凍空調技師執照與Autodesk REVTT國際認證」，並在2018年與指導李魁鵬一同研發AI人工智慧，打造節省能源動態決策系統。
郭台銘聽到這位學子的研發內容，立刻感興趣的詢問怎麼讓空調節能，因為現在中火每年在夏天用電尖峰最高都要開到8部機組，若能用AI節能省電，對於國家很有幫助，並霸氣喊出「三創空調包給你」，節省多少能源就分潤，並勉勵學子從第一步開始執行，因為萬丈高樓平地起。
▲郭台銘（左三）出席第三屆鴻海獎學鯨頒獎典禮。（圖／記者林敬旻攝）
來自於國立台灣體育運動大學的林姿妤為一位熱愛運動的身障生，大一時入選為國手參加亞洲身障桌球錦標賽榮獲金牌，目前積極受訓希望能擠進世界前八名，以獲取2020年身障奧運資格，林姿妤表示，「感謝鴻海獎學鯨讓他知道自己並不孤單，陪伴我向目標前進。」
身為身障生的碩二生耿美琪，因脊椎受傷導致下半身癱痪，但她依然勤奮求學而考上國立高雄科技大學資訊管理所，並考取多項電腦證照資格，同時還擔任多項志工，發起募款公益活動，提到「鴻海獎學鯨計畫」看見他們這群清貧孩子的需要，讓他們能放下懼怕，勇敢追夢。
「鴻海獎學鯨計畫」的源起是創辦人郭台銘看到台灣有許多家境困難卻因法令資格限制而無法申

▲兆豐金。（圖／兆豐銀提供）
記者吳靜君／台北報導
新的一年，外界都期待在2020開始，都有好的消息！兆豐銀所公布的信用卡權益當中，不僅卡友權益不縮水，其中利多御璽卡新卡友的部分，原本海外消費現金回饋3%僅限日韓，今（2020）年上半年海外消費全都回饋3%、不需要門檻、也沒有上限；舊卡友原本海外回饋2.3%直接加碼到2.58%。
今年信用卡權益當中，民眾最為關心就是，歐盟跨境交易手續費調降，許多銀行基於成本考量，取消免年費信用卡在歐盟的刷卡優惠，所以在歐洲刷卡回饋恐怕會縮水或者降為「0」。
但在兆豐銀所公布今年的刷卡權益當中，反而猛加海外刷卡消費權益。兆豐銀的利多御璽卡新卡友原本只有日韓消費才有回饋3%的好康，現在全面開放海外消費全3%，並且不需要門檻、回饋無上限。
另外，舊卡友的海外回饋為2.3%，也直接提高到2.58%，加大舊卡友的回饋攏絡人心：此外只要還有刷滿萬元送接送的服務，包含2月底前全新卡友刷滿1萬元就送1次接送；8月底前刷『國外機票/團費/訂房網/體驗旅遊』滿1萬，再送1次；所以最高送2次。
此外，雙幣鈦金商旅卡也加碼海外消費優惠，原本海外回饋1%，也直接加碼到1.5%，有計畫出國旅遊、海外消費的民眾可以多多考量。兆豐銀表示，其他如網購神卡「e秒刷」、保費神卡「mega 1」、棒球神卡「統一獅」優惠持續進行中。
----------------------------------------------------------------------

[12] 遠東集團策展生態園區向下紮根　手作多肉植物盆栽近距離親近自然

▲為慶祝遠東集團邁入七十周年，遠企購物中心與花蓮亞泥生態園區攜手規劃生態園區巡迴展，讓大人和小朋友在都會區就能近距離接觸大自然生態。（圖／遠企提供）
記者林淑慧／台北報導
遠東集團今年邁入七十周年，帶動生態學習向下紮根。遠東旗下遠企購物中心規劃亞泥生態園區巡迴展，除了有專業導覽員介紹六大竹節蟲品種，也有多肉植物盆栽等手作課程，展期自即日起至15日為止，讓大人和小朋友在都會區就能近距離接觸大自然生態。
展望後市，遠東集團總裁徐旭東日前對集團願景許下心願，期許集團能再有下一個七十年，但他也坦言「下一個七十年更挑戰」，因為集團前面七十年來自整個亞洲的經濟成長帶動，後面的七十年因為全球物聯網興起，加以新零售和AI等新科技的進步，致使零售業

▲台灣中油公司宣佈自今（2）日凌晨零時起，1月份桶裝瓦斯以及天然氣均不調漲，要讓民眾過個好年。（圖／《ETtoday新聞雲》資料照）
記者林淑慧／台北報導
農曆春節即將到來，經濟部長沈榮津日前宣佈油價宜「調降不調漲」，台灣中油宣佈自今（2）日凌晨零時起，1月份桶裝瓦斯以及天然氣均不調漲，這也是國內桶裝瓦斯連續四個月凍漲，要讓民眾過個好年。
台灣中油公司表示，自2日凌晨零時起，109年1月份國內家用液化石油氣（桶裝瓦斯）、工業用丙烷、丁烷及混合丙丁烷及車用液化石油氣等價格皆不予調整；國內天然氣價格亦不調漲。
近期國際液化石油氣價格持續攀升，國內液化石油氣價格依「台灣中油液化石油氣價格每月檢討調整機制」辦理，台灣中油指出，109年1月國際液化石油氣合約價 CP較上月大幅上漲，依液化石油氣計價公式計算結果，液化石油氣每公斤應調漲新臺幣5.0元。
台灣中油主管表示，在考量春節期間物價平穩下，本月價格不調漲，未調足之金額由台灣中油暫時吸收；加計108年10月及11月合計未調足之吸收金額每公斤新臺幣3.6元，累計吸收金額為每公斤新臺幣8.6元，將於日後國際價格回跌時再回補。
有關天然氣部分，依據政府核定之天然氣價格公式計算，1月份國內天然氣價格因氣源成本增加應調漲售價，惟考量1月適逢農曆春節，台灣中油配合政府穩定物價政策，109年1月份天然氣價格不調漲。各用戶別天然氣產品價格請詳參台灣中油全球資訊網公告之天然氣牌價表。
----------------------------------------------------------------------

[24] 國賓許育瑞推動集團義賣送暖　募集百萬捐贈馬偕醫院照護基金

▲仰德集團董事長許育瑞發起公益義賣，共募集100萬元全數捐贈馬偕紀念醫院長期照護基金。（圖／國賓大飯店提供）
記者林淑慧／台北報導
新年開春，企業拚公益人間送暖。仰德集團董事長許育瑞發動集團公益義賣，獲得各界熱心響應，總計募得上百萬元，並將所得全數捐給馬偕紀念醫院長期照護基金，為需要長期照護的長輩們盡一份心力。
仰德集團日前於台北國賓飯店大廳舉行「愛在國賓送溫情，關懷老人愛同行」慈善義賣，販售國賓麵包房手作麵包、點心，包括應景的「薑餅人」；聖誕限定麵包「史多倫」、「義大利水果麵包」；法式甜點「檸檬蛋糕」、「香蕉蛋糕」等熱銷商品。
國賓大飯店主管表示，董

記者陳心怡／台北報導
許多人認為，買房地產是穩健又安全的投資方式，可以投資包租套房，安養晚年。但是在《雲端最有錢》節目中，房產達人Sway卻不這麼認為，他說，假如一般人身上有一點錢，倒不如放到股市中，因為少子化等問題，房子要租出去都有難度。
財經專家阮慕驊認為，目前台灣沒有一個地方房價基期低，如果想買一間小套房當包租公、包租婆，實在不容易，台北市平均租金報酬率大約2%，扣掉地價稅、修繕費等等，恐怕沒賺錢反而倒貼。
此外，阮慕驊還提到，希望手頭資金流動性增加的人，可以考慮賣房子，從2014年到現在，目前正處於最佳賣房時機，因為現在買氣回升、價格也上揚，而且台灣房市未來走勢也不見得會更好，因此建議想賣房的人現在就能賣了。
----------------------------------------------------------------------

[35] 好囂張！詐騙分子親到銀行收現金　永豐銀：假親友、假投資騙很大

▲永豐銀提醒民眾，年終獎金、紅包入袋時，也常成為歹徒覬覦的對象，小心防詐騙。（圖／記者湯興漢攝）
記者吳靜君／台北報導
永豐銀公布今年協助民眾防堵詐騙的成績，累計今年前11個月，各分行通報防阻詐騙案有2,673萬元，各類型的案件當中，假親友的詐騙案最常見、假投資則是單筆詐騙金額最高。隨著年關將近，民眾的年終獎金、紅包常常成為歹徒覬覦的目標，民眾應該小心防詐騙。
根據永豐銀公布的資料顯示，今年前11月各分行通報、防堵的詐騙金額高達2673萬元，其中假親友的手法最為常見，連外籍移工也險受害。
永豐銀舉個案例，有在台外籍移工透過社群軟體認識海外友人，該名海外友人謊稱，要送禮物給該名移工，而包裹被外國海關扣留，需該名移工匯款支付通關費用，移工在行員的勸說之下才發現受到詐騙。
永豐銀說，假親友的詐騙的案例中，超過70%為國外匯款，險遭詐騙累計總金額逾600萬元，受害對象多為透過社群軟體，被假冒在國外的家人朋友所詐騙。但詐騙金額高者，多數是「假投資」，以「假投資」為由的詐騙案件量居次，但累計詐騙總金額高達1,400萬元，為所有詐騙案件中總金額占比最高的類型。
更有疑似詐騙集團份子直接出現在銀行等待客戶交付現金，因分行主管察覺有異立即通報警方處理，成功防堵客戶受騙並協助警方後續調查。
除警方通報機制外，永豐銀行內部更重視定期宣導防詐騙流程、進行防詐騙及

▲當台灣進入「超高齡社會」，民眾該如何準備自身的保障？（圖／pixabay）
記者楊絡懸／台北報導
台灣即將在2026年步入「超高齡社會」，如果少子化問題沒有改善，那麼青壯年人口未來將扛起照顧老人及同時照顧小孩的壓力，因此，壽險業者建議，應趁現在藉由保險轉嫁疾病或意外產生的困境，至少身體不方便行動時還能利用保險金過生活，減輕經濟負擔。
錠嵂保險經紀人表示，從人口結構推斷，台灣到了2026年步入「超高齡社會」，少子化問題影響的層面將非常巨大，從教育、勞動、家庭生活、消費型態至房地產，都將受到衝擊。
根據國家發展委員會提供的數據，明年65歲以上的老人佔台灣總人口16%，15至64歲的青壯年佔71%，剩下即是幼年人口，意思是每4位青壯年就得負責照顧1位老人；而30年後，2050年時，1.5個工作人口就得負責1位老人，有小孩的家庭還得同時照顧小孩。
▲保險專家認為，要善加利用保險轉嫁疾病或意外的風險，避免增加家庭負擔。（圖／pixabay）
錠嵂保經表示，在這情況下，假如老人無法行動、必須時時有人照護，那不僅身為家庭支柱的家人得賠上時間與心力，連家裡的幼童也可能被影響，無法好好成長，身為家庭支柱更不能倒下。
錠嵂保經表示，人們到了晚年，往往身體一發生狀況，想留給孩子的資源，轉眼間就花完，計畫與心血都白費掉，相當可惜，因此，提早把保險規劃得好，那孩子就是受益人；反之，若無半點計策或安排錯誤，孩子或許就成了苦不堪言的受害人。
▲「失能險」保障什麼？（圖／記者楊絡懸製表）
【延伸閱讀】
----------------------------------------------------------------------

[47] 5G競標31日達841.7億！2020年朝900億邁進

----------------------------------------------------------------------

[48] 華映破產後遺症！墊償基金打破紀錄　今撥款7.84億元給1902名員工

▲華映破產墊償基金刷新紀錄今撥款7.84億元。（圖／記者湯興漢攝）
記者余弦妙／台北報導
大同旗下子公司華映宣布破產後，因其未依法給付員工薪資及資遣費，勞動部勞保局今（31）日表示，啟動「墊償基金」，核定共1902名勞工、共撥款7.84億元，勞工刷存摺就能看見入帳，且

▲順藥總經理林榮錦。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
順天醫藥生技（6535）今（31）日宣布，旗下治療尿毒性搔癢新藥LT5001，獲得台灣衛生福利部食品管理署（TFDA）審核通過，得在台灣進行一期併二期（Phase Ib/II）人體臨床試驗，順藥預計以1年時間完成Phase Ib臨床試驗，未來可望造福因罹患慢性腎臟，而導致中重度尿毒搔癢的病患。
順藥表示，全球洗腎病患眾多，以全球最大醫藥市場美國為例，根據美國腎臟協會統計，每7位美國人就有一位罹患慢性腎臟病，全美約有370萬慢性腎臟病患(Chronic kidney disease, CKD)，而在2016年，美國有50萬人洗腎，其中尿毒性瘙癢就是慢性腎臟病與末期腎臟病患者常見的併發症。
順藥指出，根據一項名為DOPPS的跨國且持續追蹤的研究計畫（Dialysis Outcomes and Practice Patterns Study，中文：透析結果與實施類型研究），約有4成左右的洗腎病友，受中度至重度的尿毒搔癢所苦。
順藥說明，尿毒性搔癢具有強烈的「未被滿足的醫療需求」（unmet medical needs），目前治療選擇包括抗組織胺類藥品、類固醇藥品、GABA 結構類似物與保濕產品或照光治療等療法，然而對多數病患而言，或僅能短暫緩解，或效果不理想，或有嗜睡等副作用，或因副作用無法長期使用，皆無法理想解決病患的不適，導致尿毒性搔癢嚴重影響洗腎病友的生活品質。
順藥補充，LT5001為順藥開發的外用劑型新藥，塗抹在患處皮膚上，可阻斷搔癢的周邊神經訊息傳遞，達到解除或大幅緩解搔癢的效果，而且根據動物試驗，LT5001至少有4小時的止癢效果，並已證實具有約10%的經皮吸收率。
順藥強調，LT5001獲得臨床試驗許可後，將儘速規畫執行臨床試驗所需準備工作，例如醫院倫理委員會（IRB）申請等，估計在正式收案後，將依照兩個階段之試驗設計完成一期併二期的臨床試驗，預計將先一期，並依試驗結果調整執行二期，以完成整個一期併二期的臨床試驗，以驗證其初步療效，惟完成時間仍須以實際完成進度為準。
----------------------------------------------------------------------

[59] 大立光5000元封關　今年來漲55%、市值增2395億元

記者周康玉／台北

▲蘋果與虛擬機開發商Corellium間的侵權訴訟進一步升級。（圖／取自Corellium官網）
記者王曉敏／綜合報導
蘋果與虛擬機開發商Corellium間的侵權訴訟進一步升級。蘋果日前控告Corellium侵害版權，並稱該公司販售這些越獄工具違反了數位千禧年著作權法（DMCA）。不過，Corellium周日對此發布聲明，反批蘋果正試圖樹立消滅公開越獄的先例，並稱「蘋果公司持續妖魔化越獄，令我們深感失望」。
但對此，Corellium聲明表示，蘋果提交的最新文件對所有安全研究人員、應用程式開發人員及越獄者來說都值得擔心。
Corellium指出，蘋果聲稱Corellium「允許用戶越獄」並「允許一個或多個人存取...以開發可用於越獄的軟體」並加以販售，已違反數位千禧年著作權法（DMCA）。「換句話說，蘋果聲稱，任何提供允許他人越獄的工具，以及任何協助創建這種工具的人都違反了DMCA。」
Corellium認為，蘋果試圖以新的角度將此案做為「風向球」（trial balloon），以打擊越獄行為，並開了消滅公共越獄的先例。
其他人也看了這些...
----------------------------------------------------------------------

[69] 泰銖創6年半來新高！換匯價也走揚　換3萬少住一晚五星級飯店

▲ 泰銖紙鈔。（圖／CFP）
記者李瑞瑾／台北報導
新台幣匯率在今（31）天突破30元整數關卡，但泰銖兌美元在昨日就已率先突破、來到29元字頭，創下6年半來新高紀錄！看到台銀今天牌告的現金賣出匯價，泰銖匯價也在走高，逐漸逼近1.07元。
根據彭博即時匯率，泰銖兌美元匯率來到29.847泰銖兌1美元，今年以來升值逾7%，是2013年5月以來新高。
泰銖匯價也從波動平穩轉為走揚，台銀今天牌告的現金賣出匯價，泰銖來到1.0689元，創下12月12日（匯價1.0701元）以來的最高紀錄。
若以兌換新台幣3萬元來看，今天可換到2萬8,042元泰銖，相較2017年5月的匯價0.9159，當時可以換到3萬2,754元泰銖，相較起來就少了4,712元泰銖，約等於一晚5星級飯店的住宿費蒸發了。
--------------------------------------------------------------------

▲台灣中油表示，今年已完成仁澤3號及4號兩口地熱鑽井與產能測試，取得珍貴的井測數據、井孔影像等資料，可充分掌握仁澤地熱潛能範圍的空間分布。（圖／台灣中油提供，下同）
財經中心／綜合報導
台灣中油公司探採研究所經多年研究，已建立油田從探勘到開發的一條龍評估技術、成功應用頻譜分解分析新技術、建立環境法醫油污染鑑識技術、完成環境友善生物整治製劑的實驗室測試等；在地熱探測方面，108年與探採事業部在宜蘭仁澤地熱區完成仁澤3號及4號兩口地熱鑽井與產能測試。台灣中油表示，未來將繼續朝綜合性國際能源集團目標邁進。
台灣中油指出，一條龍評估技術已成功落實於查德礦區探勘工作，並完成奧瑞（ORYX）油田的開發計畫。繼107年獲查德政府核准奧瑞油田開發後，預計109年初將產出該礦區第一桶原油，成為台灣中油海外油礦探勘，第一個成功開發生產的案例。探研所成功應用頻譜分解分析新技術，提供油氣探勘評估時所需用到的古水道資訊，將有助於提升油氣探勘評估效能、增加探勘成功率。
探研所具有油品鑑識的專業技術，從民國97年開始協助環保署建立油污染鑑識技術，102年至106年執行「應用環境法醫技術建立市售柴油指紋圖譜調查計畫」，完整建立市售柴油鑑識技術，107年更繼續將技術推廣到汽油鑑識。一旦發現油污染，可立即對污染油品進行鑑識分析，找出污染源、迅速截斷漏源、釐清污染責任。
除此之外，成功完成環境友善生物整治製劑的實驗室測試，是探研所與國立中山大學產學合作的另一項成就，待調整配方後將進行模場試驗，增進油污染的分解效率。台灣中油表示，生物整治製劑的菌種來自煉油廠的廢水污泥，是一種自然界生存在油污中、且對油料具有強效分解能力的優勢菌種（群），生物分解技術能快速有效處理土壤及地下水油污，有助保護環境。
能源是國家經濟發展的命脈，地熱是取自地底源源不絕的熱能，乾淨又環保。台灣中油表示，今年已完成仁澤3號及4號兩口地熱鑽井與產能測試，取得珍貴的井測數據、井孔影像、地下電阻等資料，探研所建置完成仁澤的地質構造模型與裂隙滲透率模型，可充分掌握仁澤地熱潛能範圍的空間分布。
台灣中油進一步表示，為增加台灣自主能源比重、穩定油氣供應及推動友善環境，響應政府推動綠色能源政策，以自有鑽探技術積極進行探勘研究，將繼續朝「綜合性國際能源集團」的目標前進。
----------------------------------------

▲明年選股策略，法人建議，投資人可鎖定5G、記憶體、iPhone等議題。（示意圖／記者屠惠剛攝）
記者紀佳妘／台北報導
2019年封關倒數計時，台股今（30）日收跌38.22點，守穩在12000點之上。根據統計，今年以來到12月28日為止，外資對新興市場買超，印度市場奪下吸金王寶座，而台灣則名列第二，買超98.3億美元；展望2020年，法人表示，多數上市櫃公司看旺上半年業績動能，指數還有再創新高的可能。
回顧今年以來資金流向，根據彭博統計，吸金王由印度拿下，2019年至今吸金143.7億美元（約新台幣4,352億元），上周吸金3.3億美元（約新台幣99.9億元），也是連續3周吸金；其次為台灣，今年以來吸金98.3億美元（約新台幣2,997億元），上周吸金3.2億美元（約新台幣96.9億元），資金由流出轉為流入。
▲外資對亞洲股市買賣超金額，其中台股今年以來吸金98.3億美元。（圖／中信投信提供，點圖可放大）
中國信託台灣活力基金經理人周俊宏表示，由於外資上周逢聖誕假期，除了自營商之外，交易相對清淡，後市可待外資收假後觀察，不過展望2020年，基於多數上市櫃公司看旺上半年業績動能，指數還有再創新高的可能。
周俊宏指出，在選股議題上，iPhone宣布將推出平價版手機，有助於推升買氣與訂單數量，預期第1季淡季不淡，業績成長股以及產業題材性高的族群將扮演主流，建議投資人可鎖定5G、記憶體、iPhone等議題，同時也看好汽車零組件與設備相關。
另在亞洲其他國家部分，中國信託亞太多元入息平衡基金經理人陳彥羽表示，亞太地區11月製造業PMI多數呈現增長，對於2020年的經濟展望為溫和復甦，成長可期。
此外，陳彥羽說，多數亞洲國家目前本益比評價偏低，在經濟回升下，將可帶動外資資金回補，有利股市表現，相對看好澳洲、香港與菲律賓。至於大陸部份，目前可關注5G帶來電子新產品應用，如5G手機、智能家居、無線充電與網路安全升級等。
----------------------------------------------------------------------

[86] 影／2020年房市將崩跌？　房產達人不擔心「只會小跌、自住可以、投資不必」

2020年即將來臨，不過先前有傳言傳出，房市將崩跌，包括買房漲多跌少成絕響、老屋不都更不值錢、房市恐陷一蹶不振L型死魚盤等等驚恐消息。